# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [30]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [31]:
weather = pd.read_csv("../WeatherPy/weather.csv")


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

Create a heat map that displays the humidity for every city from Part 1, as in the following image:

In [32]:
weather = weather[weather['Lat'] != 0]
weather = weather.reset_index()
weather

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,rikitea,-23.1203,-134.9692,79.12,74,0,17.52,PF,1649640668
1,2,karratha,-20.7377,116.8463,89.04,27,0,8.70,AU,1649640668
2,3,baglan,36.1307,68.7083,76.50,33,100,1.10,AF,1649640669
3,4,tuktoyaktuk,69.4541,-133.0374,3.20,66,20,13.80,CA,1649640669
4,5,longyearbyen,78.2186,15.6401,10.24,56,0,23.02,SJ,1649640670
...,...,...,...,...,...,...,...,...,...,...
554,605,metro,-5.1131,105.3067,83.95,67,80,5.82,ID,1649640888
555,606,sokolskoye,57.1412,43.1589,38.61,99,100,2.57,RU,1649640889
556,607,asadabad,34.7824,48.1185,40.95,40,39,3.00,IR,1649640889
557,608,maromitsa,59.8683,48.0123,36.61,99,100,7.00,RU,1649640889


In [33]:
gmaps.configure(api_key = g_key)
location = weather[["Lat", "Lng"]].astype(float)
humidity = weather["Humidity"].astype(float)

In [45]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(location, weights=humidity, 
                                dissipating=False, max_intensity=100,
                                point_radius = 4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
# Bring in location data pairs, humidity data, define map and layer, and add layer to map








### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Create a dataframe named `hotel_df` that is the df created above only with the pertinent data.  See the prepared code below (`info_box_template`) to determine what data you will need for your final map.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels (aka "lodging") within 5000 meters.
* Utilize the Google Places API to find the first hotel search result.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# Create dataframe and empty column



In [ ]:
# Loop through dataframe and add nearby hotel result














In [ ]:
# NOTE: Do not change any of the code in this cell
# Analyze the code below for variables but these should be all columns in hotel_df

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map



# Display figure

